# 1 - Histograma de Gradientes Orientados - HOG

Descritor usado principalmente em visão computacional e aprendizado de máquina para detecção de objetos. No entanto, também podemos usar descritores de HOG para quantificar e representar a  forma  e a  textura.

O HOG possui 5 estágios apresentados abaixo:  
    - Normalizando a imagem antes da descrição. (Essa etapa é opcional, geralmente é feita a Normalização da lei de gama / potência)
    - Gradientes de computação nas direções x  e  y .  
    - Obtendo votos ponderados em células espaciais e de orientação.  
    - Contraste normalizando células espaciais sobrepostas.  
    - Coletando todos os histogramas de gradientes orientados para formar o vetor final.  
Para maior entendimento do HOG recomendo a leitura desses dois artigos: 
    1. https://www.learnopencv.com/histogram-of-oriented-gradients/
    2. https://gurus.pyimagesearch.com/lesson-sample-histogram-of-oriented-gradients-and-car-logo-recognition/

# 2 - Preparando o ambiente

1. Instalar as bibliotecas necessárias, a única que não esta entre as utilizada nas aulas é a do tensorflow:   
    pip install tensorflow
2. Execute as importações
3. Execute a função HOG
4. Siga as orientações sobre pastas e parametros a serem alterados

In [71]:
#Importações
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from skimage.io import imread,imshow
from skimage.io import imread_collection
import glob
from tensorflow import Session as s
from PIL import Image
%matplotlib inline

# 3 - Função HOG 

In [72]:
#Função do descritor HOG 
def descritor_hog(images, cell_size = 8, block_size = 2, n_bins = 9, grayscale = False):
    #---------------------------- definição de funções indiciais
    def preparacao_filtro(nucleo, n_canais):
        return np.tile(nucleo, (n_canais, 1, 1)).swapaxes(0,1).swapaxes(1,2)

    #gera gx e gy utilizando um kernel pré definido (pode ser usado sobel, etc.)
    def derivada(lote, ksize=3, padding='SAME'):
        try:
            n_ch = lote.shape[3].value
        except:
            n_ch = int(lote.get_shape()[3])
        gx = preparacao_filtro(np.array([[ 0, 0, 0],[-1, 0, 1],[ 0, 0, 0]]), n_ch)    
        gy = preparacao_filtro(np.array([[ 0, -1, 0],[ 0, 0, 0],[ 0, 1, 0]]), n_ch)   
        kernel = tf.constant(np.stack([gx, gy], axis=-1), name="DerivKernel", dtype = np.float32)
        return tf.nn.depthwise_conv2d(lote, kernel, [1, 1, 1, 1], padding=padding, name="GradXY")

    #transforma radianos em graus
    def radiano_para_grau(radiano):
        return 180 * radiano / tf.constant(np.pi)

    def select_by_idx(a, idx, grayscale):
        if grayscale:
            return a[:,:,:,0]
        else:
            return tf.where(tf.equal(idx, 2), a[:,:,:,2], tf.where(tf.equal(idx, 1), a[:,:,:,1], a[:,:,:,0]))
    #----------------------------------- fim das funções iniciais -----------------
    #Definir o tamanho do lote, altura, largura e dimensões    
    batch_size, height, width, depth = images.shape
    scale_factor = tf.constant(180 / n_bins, name="scale_factor", dtype=tf.float32)
    #Quantidade de blocos que anda
    block_stride = 1

    img = tf.constant(images, name="ImgBatch", dtype=tf.float32)

    #Condição para converter imagem para cinza
    if grayscale:
        img = tf.image.rgb_to_grayscale(img, name="ImgGray")

    # preenchimento automático de altura e largura para tamanho válido (múltiplos de tamanho de célula)
    if height % cell_size != 0 or width % cell_size != 0:
        height = height + (cell_size - (height % cell_size)) % cell_size
        width = width + (cell_size - (width % cell_size)) % cell_size
        img = tf.image.resize_image_with_crop_or_pad(img, height, width)
    
    # gradientes x e y
    grad = derivada(img)
    g_x = grad[:,:,:,0::2]
    g_y = grad[:,:,:,1::2]
    
    # mascarando gradientes indesejados de pixels de borda
    mask_depth = 1 if grayscale else depth
    g_x_mask = np.ones((batch_size, height, width, mask_depth))
    g_y_mask = np.ones((batch_size, height, width, mask_depth))
    g_x_mask[:, :, (0, -1)] = 0
    g_y_mask[:, (0, -1)] = 0
    g_x_mask = tf.constant(g_x_mask, dtype=tf.float32)
    g_y_mask = tf.constant(g_y_mask, dtype=tf.float32)
    
    g_x = g_x*g_x_mask
    g_y = g_y*g_y_mask

    # obtenção da magnitude
    g_norm = tf.sqrt(tf.square(g_x) + tf.square(g_y), "GradNorm")
    
    if not grayscale and depth != 1:
        # seleção de gradiente de norma máxima
        idx    = tf.argmax(g_norm, 3)
        g_norm = tf.expand_dims(select_by_idx(g_norm, idx, grayscale), -1)
        g_x    = tf.expand_dims(select_by_idx(g_x,    idx, grayscale), -1)
        g_y    = tf.expand_dims(select_by_idx(g_y,    idx, grayscale), -1)
    
    # obtenção da orientação (arco tangente de gy/gx)
    g_dir = radiano_para_grau(tf.atan2(g_y, g_x)) % 180
    g_bin = tf.to_int32(g_dir / scale_factor, name="Bins")

    # particionamento de células
    cell_norm = tf.space_to_depth(g_norm, cell_size, name="GradCells")
    cell_bins = tf.space_to_depth(g_bin,  cell_size, name="BinsCells")

    # histogramas de células
    hist = list()
    zero = tf.zeros(cell_bins.get_shape()) 
    for i in range(n_bins):
        mask = tf.equal(cell_bins, tf.constant(i, name="%i"%i))
        hist.append(tf.reduce_mean(tf.where(mask, cell_norm, zero), 3))
    hist = tf.transpose(tf.stack(hist), [1,2,3,0], name="Hist")

    # blocos de particionamento
    block_hist = tf.extract_image_patches(hist, ksizes  = [1, block_size, block_size, 1], strides = [1, block_stride, block_stride, 1], rates  = [1, 1, 1, 1], padding = 'VALID', name = "BlockHist")

    # normalização de blocos
    block_hist = tf.nn.l2_normalize(block_hist, 3, epsilon=1.0)
    
    # HOG descriptor
    descritor_hog = tf.reshape(block_hist, 
                                [int(block_hist.get_shape()[0]), 
                                 int(block_hist.get_shape()[1]) * \
                                 int(block_hist.get_shape()[2]) * \
                                 int(block_hist.get_shape()[3])], 
                                 name='HOGDescriptor')
    
    
    return descritor_hog

# Função para criação dinamica de matriz
def cria_matrix(tam):
    m = []
    for i in range(tam):
        m.append([])
    return m

def converte_array(matriz):
    linhas = len(matriz)
    _,col = matriz[0][0].shape
    ultima = col
    new_matriz = np.zeros((linhas,col+1))
    for i in range(linhas):
        for j in range(col):
            new_matriz[i][j] = matriz[i][0][0,j]
        new_matriz[i][ultima] = matriz[i][1]
    return new_matriz

# 4 - UTILIZANDO O HOG 

#### 1. Defina sua pasta raiz
#### 2. Dentro da pasta raiz deve conter duas pastas: maligno e benigno. (favor alterar a extensão dos arquivos para o adequado)
#### 3. Nome do arquivo CSV
## Parametros do HOG para diversas combinações (celulas e blocos)
#### 4. Numero de celulas
#### 5. Tamanho do bloco
## (ALTERE PARAMETROS APENAS AQUI)

In [80]:
#Pasta raiz
path_database = 'base_melanoma/'
#Pastas das classes (maligno e benigno) / NÃO SE ESQUEÇA DE ADEQUAR A EXTENSÃO DO ARQUIVO
imagens_maligno = imread_collection(path_database+'maligno/*.jpeg')
imagens_benigno = imread_collection(path_database+'benigno/*.jpg')
#Nome do arquivo csv que será salvo para a combinação de parametros
nome_do_csv = 'combinacao1'

#Dois principais parametros do HOH
#Numero de celulas (obs.: enquanto menor, maior sera seu vetor de caracteristicas)
cell_size = 16
#Numero de blocos
block_size = 4

# SALVAR O CSV 
### Execute o codigo abaixo para salvar o arquivo csv com as caracteristicas extraidas

In [74]:
tamanho_maligno = len(imagens_maligno)
tamanho_benigno = len(imagens_benigno)
tam = tamanho_maligno+tamanho_benigno
matriz = cria_matrix(tam)

#Iniciando sessão tensorflow
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    #Aplicacao na base maligna
    for i,imagem in enumerate(imagens_maligno):
        l,a,dim = imagem.shape
        new_imagem = imagem.reshape([-1,l,a,3])/255
        hog_descriptor = descritor_hog(new_imagem,cell_size=16,block_size=4, grayscale = True)
        hog = hog_descriptor.eval()
        matriz[i].append(hog)
        matriz[i].append(1)
    # Aplicacao na base benigna
    for i,imagem in enumerate(imagens_benigno,start=tamanho_maligno):
        l,a,dim = imagem.shape
        new_imagem = imagem.reshape([-1,l,a,3])/255
        hog_descriptor = descritor_hog(new_imagem,cell_size=16,block_size=4, grayscale = True)
        hog = hog_descriptor.eval()
        matriz[i].append(hog)
        matriz[i].append(0)

# Matriz Resultante
nova_matriz = converte_array(matriz)
# salvando em csv
np.savetxt(nome_do_csv,nova_matriz,delimiter=",")

### Sugestão para visualizar os dados do csv

In [78]:
#Para visualizar de forma bonita utilize o pandas, para isso descomente as linhas seguintes
#import pandas as pd
#seus_dados = pd.read_csv(nome_do_csv, sep=',',header=None)

In [79]:
#seus_dados.shape
#seus_dados